# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI
import gradio as gr

In [27]:
# Initialize and constants

load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')
ollama_url = os.getenv('OLLAMA_BASE_URL')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gemma3:12b'
openai = OpenAI(base_url=ollama_url, api_key='lmao')

# MODEL = 'gpt-5-nano'
# openai = OpenAI()

In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [15]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [16]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'proficient page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'projects page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'offerings page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog index', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/11/11/ai-live-event/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-

In [9]:
select_relevant_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'about page', 'url': 'https://huggingface.co/brand'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

In [10]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [19]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling llama3.2
Found 4 relevant links


{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'Company page', 'url': 'https://edwarddonner.com/'},
  {'type': 'Careers/Jobs page',
   'url': 'https://www.linkedin.com/in/eddonner/'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 6 relevant links


{'links': [{'type': 'About page', 'url': 'https://huggingface.co/'},
  {'type': 'Company page', 'url': 'https://brand.huggingface.co/'},
  {'type': 'Careers/Jobs page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'Blog', 'url': 'https://blog.huggingface.co/'},
  {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'},
  {'type': 'Discord channel', 'url': 'https://join.discord.com/HuggingFace'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [28]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [29]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gemma3:12b
Found 14 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
zai-org/GLM-Image
Updated
4 days ago
•
6.64k
•
840
Lightricks/LTX-2
Updated
about 16 hours ago
•
1.54M
•
1.16k
fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
Updated
12 days ago
•
53.1k
•
755
openbmb/AgentCPM-Explore
Updated
about 18 hours ago
•
1.55k
•
348
google/translategemma-4b-it
Updated
4 days ago
•
12.8k
•
319
Browse 2M+ models
Spaces
Running
on
Zero
Featured
939
Qwen Image Multiple Angles 3D Camera
🎥
939
Adjust camera angles in images using 3D controls or sliders
Running
on
Zero
MCP
1.59k
Z Image Turbo
🖼
1.59k
Generate stunnin

In [30]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [31]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [32]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemma3:12b
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nzai-org/GLM-Image\nUpdated\n4 days ago\n•\n6.64k\n•\n840\nLightricks/LTX-2\nUpdated\nabout 16 hours ago\n•\n1.54M\n•\n1.16k\nfal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA\nUpdated\n12 days ago\n•\n53.1k\n•\n755\nopenbmb/AgentCPM-Explore\nUpdated\nabout 18 hours ago\n•\n1.55k\n•\n348\ngoogle/translategemma-4b-it\nUpdated\n4 days ago\n•\n12.8k\n•\n319\nBrowse 2M+ models\nSpaces\nRunning\non\n

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="llama3.2",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 6 relevant links


Hugging Face Brochure
======================

Welcome to Hugging Face, the pioneering platform that's shaping the future of Artificial Intelligence.

**Mission**

Hugging Face is dedicated to building a community of machine learning enthusiasts who collaborate on model development, dataset creation, and application-building. Our mission is to accelerate innovation in AI by providing an open-source collaboration platform.

**What We Do**

We host unlimited public models, datasets, and applications, giving users the freedom to explore and build on each other's work. Our platform supports various modalities, including text, image, video, audio, and even 3D data.

**Key Features**

- **Unlimited Models**: Browse over 2 million models created by our community.
- **Collaboration Platform**: Work with others on public models, datasets, and applications.
- **Open-Source Stack**: Move faster with our free, open-source technology.
- **Variety of Modalities**: Explore text, image, video, audio, or 3D data.

**Community**

Our community is at the heart of Hugging Face. We believe that collaboration is key to driving innovation in AI. Join us and become part of a vibrant ecosystem that's pushing the boundaries of what's possible with machine learning.

**Careers and Jobs**

Are you interested in joining our mission? We're always looking for talented individuals to contribute to our platform. Check out our [careers page](link) to learn more about opportunities at Hugging Face.

**Pricing**

Our platform is free, but we offer paid plans for those who need additional features or support.

Join the Hugging Face community today and be a part of shaping the future of AI!

Contact Us
----------

Want to get in touch with us? Visit our [about page](link) to learn more about our company and mission.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [34]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemma3:12b
Found 14 relevant links


**Welcome to Hugging Face: The Future of Machine Learning**

At Hugging Face, we're passionate about bringing together the machine learning community to build the future. Our platform is more than just a collection of models and datasets – it's a hub for collaboration, innovation, and creativity.

**Building Together**

We believe that machine learning should be accessible to everyone, regardless of background or expertise. That's why we've created a collaborative space where anyone can host and share their own models, datasets, and applications. Whether you're a researcher, entrepreneur, or student, our platform provides the tools and resources you need to build your portfolio and showcase your work.

**Unlimited Possibilities**

With over 2 million models and 500,000+ applications available, the possibilities are endless. From image-to-image translation to text-to-speech synthesis, we've got you covered. Our platforms support multiple modalities, including text, images, video, audio, and even 3D.

**Accelerating Your ML**

At Hugging Face, we're committed to making machine learning more accessible and affordable. That's why we offer a range of resources, including paid computation services and open-source software stack. Our team of experts is always here to help you navigate the latest developments in machine learning.

**Join the Community**

What sets us apart is our community-driven approach. We're not just another machine learning platform – we're a family. Join us today and be part of the movement that's shaping the future of AI.

**Careers at Hugging Face**

We're looking for talented individuals to join our team. If you're passionate about machine learning, collaboration, and innovation, we'd love to hear from you. Check out our careers page for more information on available positions.

---

**Get Started**

Ready to explore the future of machine learning? Sign up now for free access to our platform and start building your portfolio today!

[button: Sign Up]

---

Follow us on social media to stay up-to-date with the latest developments in Hugging Face:

Twitter | LinkedIn | Facebook

In [33]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling llama3.2
Found 5 relevant links


# Welcome to Hugging Face

We're thrilled to have you here! At Hugging Face, we're building a community of machine learning enthusiasts who collaborate on the future of AI. Whether you're just starting out or already an expert in the field, we invite you to join our ecosystem.

## Our Mission

We believe that collaboration is key to unlocking human potential. That's why we created a platform where anyone can create, discover, and contribute to machine learning models, datasets, and applications.

## What We Offer

* **2M+ Models**: Browse over 2 million public models developed by our community.
* **Collaboration Platform**: Host and collaborate on unlimited public models, datasets, and applications.
* **Open Source Stack**: Move faster with our open source stack and explore all modalities – text, image, video, audio, or even 3D!
* **Build Your Portfolio**: Share your work with the world and build your ML profile.

## Our Community

Our community is passionate about machine learning and AI. We share knowledge, resources, and expertise to help each other grow. Join our forums and discussions on various topics, from beginner-friendly guides to research papers and more!

### Hugging Face Forums

* **Beginners' Corner**: Get started with AI and ML tutorials, advice, and support.
* **Research Discussions**: Engage with experts and researchers on cutting-edge topics like LLMs, Natural Language Processing (NLP), and more!
* **Success Stories**: Share your achievements and learn from others in the community.

## Join Us!

At Hugging Face, we're committed to accelerating your machine learning journey. Whether you're just starting out or looking for opportunities to collaborate and grow, join our community today!

---

**Hugging Face Careers**

Ready to be part of a dynamic team that's shaping the future of AI? Check out our job listings and careers pages to explore opportunities in:

* **Machine Learning Engineering**: Develop and deploy cutting-edge models with our community.
* **Research and Development**: Contribute to breakthroughs in NLP, computer vision, and more.
* **Customer Support**: Help users navigate our platform and answer questions.

Join us and help build the future of AI together!

---

**Hugging Face Enterprise**

Looking for a customized solution for your organization? Our enterprise solutions combine the power of Hugging Face models with personalized support and integration. Contact us to learn more about how we can help you accelerate your AI journey.

---

We're glad you're here!

In [43]:
company_input = gr.Textbox(label="Company name:")
link_input = gr.Textbox("https://", label="Company link:")

brochure = gr.Markdown(label='Response:')

gr.Interface(
    fn=stream_brochure,
    inputs=[company_input, link_input],
    outputs=[brochure],
    examples=[
        ['HuggingFace', 'https://huggingface.co'],
        ['CNN', 'https://cnn.com']
    ],
    flagging_mode='never'
).launch()

* Running on local URL:  http://127.0.0.1:7877
* To create a public link, set `share=True` in `launch()`.


Selecting relevant links for https://huggingface.co by calling gemma3:12b
Found 13 relevant links


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>